# Data Processing

1. Read .csv files
2. Clean tables
3. Merge Tables
4. Feature Selection

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import date as d
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### Utils functions

In [2]:
def read_file(file_name, date=False):
    return pd.read_csv('data/' + file_name + '.csv', sep=';', parse_dates=date, na_values=['NA', ''], low_memory=False).rename(str.strip, axis = 'columns')

In [3]:
# Convert dates in yymmdd format to yyyy-mm-dd
def parse_date(date):
    return pd.to_datetime('19'+str(date)[0:2] + str(date)[2:4] + str(date)[4:6])

# Returns current age
def calc_age(date):
    today = d.today()
    return today.year - date.year - ((today.month, today.day) < (date.month, date.day))

Read files, clean data and select features

In [4]:
def process_account():
    account_df = read_file('account', ['date'])
    
    # Calculate account age
    account_df['date'] = account_df['date'].apply(lambda date: calc_age(date))
    account_df.rename(columns={'date':'age_acc'}, inplace=True)
    
    le = LabelEncoder()
    account_df['frequency'] = le.fit_transform(account_df['frequency'])
    
    return account_df

In [5]:
def process_card(type_d):
    card_df = read_file('card_'+type_d, ['issued'])
    
    card_df.rename(columns={'type':'type_card'}, inplace=True)
    
    # issued date in a better format
    #card_df['issued'] = card_df['issued'].apply(lambda date: parse_date(date))
    
    # Remove issued date
    card_df.drop(columns='issued', inplace=True)
    
    return card_df

In [6]:
def process_client():
    client_df = read_file('client')
    
    # Extract client's age and gender from birthnumber
    ages = []
    genders = []
    for bn in client_df['birth_number']:
        month = int(str(bn)[2:4])
    
        # Gender
        if month > 12:
            genders.append(0) # Female
            bn -= 5000
        else:
            genders.append(1) # Male

        # Age
        ages.append(calc_age(parse_date(bn)))
    
    client_df['age_clt'] = ages
    client_df['gender_clt'] = genders
    client_df.drop(columns='birth_number', inplace=True)
    
    return client_df

In [7]:
def process_disposition():
    disp_df = read_file('disp')
    
    # Only owners can ask for loans
    disp_df = disp_df.loc[disp_df.type == 'OWNER']
    disp_df = disp_df.drop(labels='type', axis=1)
    disp_df.rename(columns={'type':'type_disp'}, inplace=True)
    
    return disp_df

In [8]:
def process_district():
    district_df = read_file('district')
    
    district_df.drop(columns='name', inplace=True)
    
    le = LabelEncoder()
    district_df['region'] = le.fit_transform(district_df['region'])
    
    # Assign '96 values to missing values' cells from '95
    district_df["unemploymant rate '95"] = np.where(district_df["unemploymant rate '95"] == '?', district_df["unemploymant rate '96"], district_df["unemploymant rate '95"])
    district_df["no. of commited crimes '95"] = np.where(district_df["no. of commited crimes '95"] == '?', district_df["no. of commited crimes '96"], district_df["no. of commited crimes '95"])
    district_df["unemploymant rate '95"] = pd.to_numeric(district_df["unemploymant rate '95"])
    district_df["no. of commited crimes '95"] = pd.to_numeric(district_df["no. of commited crimes '95"])
    
    district_df.rename(columns={'code': 'district_id'}, inplace=True)
    
    return district_df

In [9]:
def process_loan(type_d):
    loan_df = read_file('loan_'+type_d, ['date'])
    
    loan_df.rename(columns={'amount': 'amount_loan'}, inplace=True)
    loan_df.rename(columns={'date':'date_loan'}, inplace=True)
    #loan_df.drop(columns='date', inplace=True)
    
    return loan_df

In [10]:
def process_trans(type_d):
    trans_df = read_file('trans_'+type_d, ['date'])
    
    # When the Operation is Null k_symbol has info
    trans_df.loc[trans_df['operation'].isna(),'operation'] = trans_df.loc[trans_df['operation'].isna(),'k_symbol']
    
    # Convert 'withdrawal in cash' to 'withdrawal' in type
    trans_df.loc[trans_df['type']=='withdrawal in cash','type'] = 'withdrawal'
    
    trans_df.rename(columns={'type': 'type_trans', 'amount': 'amount_trans'}, inplace=True)
    #trans_df.rename(columns={'date':'date_trans'}, inplace=True)
    trans_df.drop(columns=['date', 'k_symbol', 'bank', 'account'], inplace=True) #date
    
    return trans_df

#### Merge data

In [11]:
def process_merge_data(type_d):
    account_df = process_account()
    card_df = process_card(type_d)
    client_df = process_client()
    disp_df = process_disposition()
    district_df = process_district()
    loan_df = process_loan(type_d)
    trans_df = process_trans(type_d)
    
    data = loan_df.merge(disp_df, on='account_id')
    data = data.merge(client_df, on='client_id')
    data = data.merge(account_df, on='account_id', suffixes=('_clt', '_acc'))
    data = data.merge(district_df, left_on='district_id_clt', right_on='district_id')
    data = data.merge(trans_df, on='account_id')
    #data = data.merge(card_df, on='disp_id')
    
    data = data.drop(columns={'account_id', 'disp_id', 'district_id_clt', 'district_id_acc', 'district_id', 'client_id', 'trans_id',
                              'no. of municipalities with inhabitants < 499',
                              'no. of municipalities with inhabitants 500-1999', 
                              'no. of municipalities with inhabitants 2000-9999',
                              'no. of municipalities with inhabitants >10000',
                              'no. of cities',
                              'ratio of urban inhabitants'
                             })
    
    return data

#### Get Train and Test Data

In [12]:
train_data = process_merge_data('train')
test_data = process_merge_data('test')

print("<Train Data>")
print(train_data.info())
print("\n<Test Data>")
print(test_data.info())

<Train Data>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24494 entries, 0 to 24493
Data columns (total 22 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   loan_id                                    24494 non-null  int64         
 1   date_loan                                  24494 non-null  datetime64[ns]
 2   amount_loan                                24494 non-null  int64         
 3   duration                                   24494 non-null  int64         
 4   payments                                   24494 non-null  int64         
 5   status                                     24494 non-null  int64         
 6   age_clt                                    24494 non-null  int64         
 7   gender_clt                                 24494 non-null  int64         
 8   frequency                                  24494 non-null  int32         
 9   age_

#### Data aggregation

In [13]:
def abs_min(x):
    return x.abs().min()
def rangev(x):
    return x.max() - x.min()

def count_withdrawal(x):
    return sum(x=='withdrawal')
def count_credit(x):
    return sum(x=='credit')

def mean_withdrawal(x):
    return np.mean(x=='withdrawal')
def mean_credit(x):
    return np.mean(x=='credit')

def cov_withdrawal(x):
    return np.cov(x=='withdrawal')
def cov_credit(x):
    return np.cov(x=='credit')

In [14]:
def aggregate_data(df):
    # Keep all columns except the ones we are aggregating
    keep_columns = train_data.columns.to_list()
    keep_columns.remove('type_trans')
    keep_columns.remove('operation')
    keep_columns.remove('amount_trans')
    keep_columns.remove('balance')

    df = df.groupby(keep_columns, as_index=False, group_keys=False).agg({
        'type_trans': [count_withdrawal, count_credit, mean_withdrawal, mean_credit, cov_withdrawal, cov_credit],
        'operation': ['count'],
        'amount_trans': ['mean','min','max','std','last', abs_min],
        'balance': ['mean','min','max','std','last', abs_min]
    })
    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]
    
    return df

In [15]:
train_data_agg = aggregate_data(train_data)

# we should sort by loan data to have a realistic train/test split while training
train_data_agg.sort_values(by=['date_loan'], inplace=True)
train_data_agg.drop(columns='date_loan', inplace=True)

# XGBoost need labels from 0...[num_class -1], also makes status a 'boolean', to loan or not
train_data_agg.loc[train_data_agg['status'] == -1, 'status'] = 0

test_data.status = test_data.status.fillna('')
test_data_agg = aggregate_data(test_data)

print("<Train Data>")
print(train_data_agg.info())
print("\n<Test Data>")
print(test_data_agg.info())

<Train Data>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 38 to 327
Data columns (total 36 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   loan_id                                    328 non-null    int64  
 1   amount_loan                                328 non-null    int64  
 2   duration                                   328 non-null    int64  
 3   payments                                   328 non-null    int64  
 4   status                                     328 non-null    int64  
 5   age_clt                                    328 non-null    int64  
 6   gender_clt                                 328 non-null    int64  
 7   frequency                                  328 non-null    int64  
 8   age_acc                                    328 non-null    int64  
 9   region                                     328 non-null    int64  
 10  no. of inhab

**ToDo** : Pôr os ficheiros de baixo dentro de uma pasta `model_data`

In [16]:
train_data_agg.to_pickle('train_data.pkl')
test_data_agg.to_pickle('test_data.pkl')